Create summary of the requested runs.

In [29]:
import os
import json
import yaml
import numpy as np
from pathlib import Path
from tensorboard.backend.event_processing.event_file_loader import EventFileLoader
from collections import OrderedDict

In [30]:
from_run = 6107
to_run = 6108

In [31]:
def get_run_dict(paths_dict: dict) -> dict:
    result = {}
    #dir_contents: list = []
    for key in paths_dict:
        if key != "working_dir":
            dir_contents = os.listdir(paths_dict[key])
    
            #numbers = []
            #rest_of_file_name: str = ""
            for entry in dir_contents:
                number = 0
                if "_" in entry:
                    number = int(entry.split("_")[0])
                    #rest_of_file_name = str.join("_", entry.split("_")[1:])
                    
                else:
                    number = int(entry.split(".")[0])
                    #rest_of_file_name = str.join(".", entry.split(".")[1:])
                    
                if number not in result:
                    result[number] = {}
                    
                result[number][key] = paths_dict[key] / f"{entry}"
    #if not numbers:
    #    return []
    
            #for num in numbers:
                
    
    # Remove double entries and sort.
    #result = dict.fromkeys(numbers)
    #result.sort()

    #for key in result:
    #    result[key] = {}
    #    result[key]["summary_path"] = paths_dict["summaries_dir"] / f"{key}_summary.json"
    #    result[key]["result_path"] = paths_dict["summaries_dir"] / f"{key}_summary.json"

    return result

In [32]:
def get_result_overview(path: Path, unity: dict) -> dict:
    """Get the mean reward over the last 5 cumulative rewards entries in the tfevents file."""
    cumulative_rewards = []
    ep_length = []

    # Get the tfevents file associated with the current run.
    path_to_result_folder = path / "RollerAgent/"
    try:
        path_to_result = sorted(Path(path_to_result_folder).glob("events.out.tfevents.*"))[0]
    except:
        return {}

    # Using tensorflow to access the tfevents data.
    # datarecord = EventFileLoader(str(path_to_result)).Load()
    for event in EventFileLoader(str(path_to_result)).Load():
        # event = event_pb2.Event.FromString(batch.numpy())
        for value in event.summary.value:
            if value.tag == "Environment/Cumulative Reward":
                cumulative_rewards.append(value.tensor.float_val[0])
            if value.tag == "Environment/Episode Length":
                ep_length.append(value.tensor.float_val[0])

    # Return mean of the last 5 recorded cummulative rewards.
    rewards_of_interest = cumulative_rewards[-100:]
    return {"mean_reward": np.mean(rewards_of_interest), 
            "std_mean_reward": np.round(np.std(rewards_of_interest), 5),
            "episode_length": np.mean(ep_length[-100:]),
            "normalised_reward": np.mean(rewards_of_interest) / unity["maxStep"]
    }

In [33]:
# Enter paths for plot creation and get absolute paths.
paths = {
        "working_dir": "C:/Users/max.muehlefeldt/Documents/GitHub/unity-machine-learning/python/basic_rl_env",
        "results_dir": "results/",
        "results_archive_dir": "results_archive/",
        "stats_dir": "stats/",
        "summaries_dir": "summaries/",
        "configs_dir": "configs/",
        "unity_configs_dir": "unity_configs/"
    }

paths["working_dir"] = Path(paths["working_dir"]).absolute()

for key in paths:
    if key != "working_dir":
        paths[key] = paths["working_dir"] / paths[key]
        

paths

{'working_dir': WindowsPath('C:/Users/max.muehlefeldt/Documents/GitHub/unity-machine-learning/python/basic_rl_env'),
 'results_dir': WindowsPath('C:/Users/max.muehlefeldt/Documents/GitHub/unity-machine-learning/python/basic_rl_env/results'),
 'results_archive_dir': WindowsPath('C:/Users/max.muehlefeldt/Documents/GitHub/unity-machine-learning/python/basic_rl_env/results_archive'),
 'stats_dir': WindowsPath('C:/Users/max.muehlefeldt/Documents/GitHub/unity-machine-learning/python/basic_rl_env/stats'),
 'summaries_dir': WindowsPath('C:/Users/max.muehlefeldt/Documents/GitHub/unity-machine-learning/python/basic_rl_env/summaries'),
 'configs_dir': WindowsPath('C:/Users/max.muehlefeldt/Documents/GitHub/unity-machine-learning/python/basic_rl_env/configs'),
 'unity_configs_dir': WindowsPath('C:/Users/max.muehlefeldt/Documents/GitHub/unity-machine-learning/python/basic_rl_env/unity_configs')}

In [34]:
run_dict = get_run_dict(paths)

selected_ids = [x for x in run_dict.keys() if from_run <= x <= to_run]
selected_ids




[6107, 6108]

In [35]:
selected_ids_dict = {}
for run_id in selected_ids:
    selected_ids_dict[run_id] = {}

    if "stats_dir" in run_dict[run_id]:
        with open(run_dict[run_id]["stats_dir"]) as json_file:
        #results["stats"] = json.load(json_file)
            selected_ids_dict[run_id]["stats"] = json.load(json_file)
    
    if "unity_configs_dir" in run_dict[run_id]:
        with open(run_dict[run_id]["unity_configs_dir"]) as json_file:
        #results["stats"] = json.load(json_file)
            selected_ids_dict[run_id]["unity_config"] = json.load(json_file)

    if "configs_dir" in run_dict[run_id]:
        with open(run_dict[run_id]["configs_dir"]) as yaml_file:
        #results["stats"] = json.load(json_file)
            selected_ids_dict[run_id]["training_config"] = yaml.safe_load(yaml_file)
    
    if "results_dir" in run_dict[run_id]:
        selected_ids_dict[run_id]["result"] = get_result_overview(run_dict[run_id]["results_dir"], selected_ids_dict[run_id]["unity_config"])


selected_ids_dict

{6107: {'stats': {'sameRoom': 467691,
   'agentInRoomID0': 467950,
   'agentInRoomID1': 467338,
   'targetInRoomID0': 468137,
   'targetInRoomID1': 467151,
   'episodeCount': 935288,
   'sensorCount': 32,
   'runId': 6107,
   'targetRoomIndex0': 468137,
   'targetRoomIndex1': 467151},
  'unity_config': {'sensorCount': 32,
   'useDecoy': False,
   'createWall': True,
   'doorWidth': 4.0,
   'randomWallPosition': True,
   'randomDoorPosition': True,
   'targetAlwaysInOtherRoomFromAgent': False,
   'targetFixedPosition': False,
   'maxStep': 1000,
   'stepPenalty': -0.0005,
   'runId': 6107,
   'statsExportPath': 'C:\\Users\\max.muehlefeldt\\Documents\\GitHub\\unity-machine-learning\\python\\basic_rl_env\\stats\\6107_stats.json'},
  'training_config': {'behaviors': {'RollerAgent': {'hyperparameters': {'batch_size': 356,
      'beta': '1e-2',
      'buffer_size': 51200,
      'epsilon': 0.2,
      'lambd': 0.95,
      'learning_rate': '1e-3',
      'learning_rate_schedule': 'linear',
     

In [36]:
#selected_ids_dict[6183]p.pop()

In [37]:
# Remove ids from the dict if no result in dict saved.
problem_ids = []
for entry in selected_ids_dict:
    if "result" not in selected_ids_dict[entry]:
        problem_ids.append(entry)

for id in problem_ids:
    selected_ids_dict.pop(id)


In [38]:
sorted_dict = OrderedDict(
            sorted(
                selected_ids_dict.items(), key=lambda v: v[1]["result"]["mean_reward"], reverse=True
            )
        )
sorted_dict

OrderedDict([(6108,
              {'stats': {'sameRoom': 754842,
                'agentInRoomID0': 754758,
                'agentInRoomID1': 753953,
                'targetInRoomID0': 754649,
                'targetInRoomID1': 754062,
                'episodeCount': 1508711,
                'sensorCount': 32,
                'runId': 6108,
                'targetRoomIndex0': 754649,
                'targetRoomIndex1': 754062},
               'unity_config': {'sensorCount': 32,
                'useDecoy': False,
                'createWall': True,
                'doorWidth': 4.0,
                'randomWallPosition': True,
                'randomDoorPosition': True,
                'targetAlwaysInOtherRoomFromAgent': False,
                'targetFixedPosition': False,
                'maxStep': 1000,
                'stepPenalty': -0.0005,
                'runId': 6108,
                'statsExportPath': 'C:\\Users\\max.muehlefeldt\\Documents\\GitHub\\unity-machine-learning\\python\\b

In [39]:
sorted_dict[6202]

KeyError: 6202

In [ ]:
all_normalised_reward = []

for key in sorted_dict:
    all_normalised_reward.append(sorted_dict[key]["result"]["normalised_reward"])

np.mean(all_normalised_reward)
#all_normalised_reward

-0.001524721241281006